# 必要な準備を行う

ワークフロー実行のために必要なGINのユーザー情報を保持します。  
上から順番に実行してください。

## ◆◆◆開発メモ◆◆◆
ゆくゆくはSSOに対応し、コンテナ立ち上げ毎のユーザー名、パスワード、メールアドレスの入力を避けたい

## 1.ユーザー情報を情報を入力する

以下のセルを実行してデータガバナンス機能のユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
from IPython.display import clear_output
import getpass
name = input("ユーザー名：")
password = getpass.getpass("パスワード：")
email = input("メールアドレス：")
clear_output()

## 2.ユーザー情報を登録する

### .netrcにユーザー情報を追加する

In [ ]:
!echo -e "machine dg02.dg.rcos.nii.ac.jp\nlogin $name\npassword $password" > ~/.netrc